In [2]:
import zipfile
import os
import sys

for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames[:10]:
        print(os.path.join(dirname, filename))
#sys.exit()
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip',"r") as z:
    z.extractall()    
for dataset in ["train","test"]:
    for label in [0,1]:
        try:
            os.makedirs("/kaggle/working/dataset/{}/{}".format(dataset,label))
        except OSError as e:
            print(e)
os.makedirs("/kaggle/working/out")

os.system("mv /kaggle/working/train/dog* /kaggle/working/dataset/train/0/.")
os.system("mv /kaggle/working/train/cat* /kaggle/working/dataset/train/1/.")

print(os.system("ls ./dataset/train/0 | shuf | head -n 1200 | xargs -I % mv /kaggle/working/dataset/train/0/% /kaggle/working/dataset/test/0/"))
print(os.system("ls ./dataset/train/1 | shuf | head -n 1200 | xargs -I % mv /kaggle/working/dataset/train/1/% /kaggle/working/dataset/test/1/"))


for dirname, _, filenames in os.walk('/kaggle/working/dataset'):
    print(dirname,len(filenames))
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/__notebook_source__.ipynb


shuf: write error: Broken pipe
shuf: write error


0


shuf: write error: Broken pipe
shuf: write error


0
/kaggle/working/dataset 0
/kaggle/working/dataset/train 0
/kaggle/working/dataset/train/1 11300
/kaggle/working/dataset/train/0 11300
/kaggle/working/dataset/test 0
/kaggle/working/dataset/test/1 1200
/kaggle/working/dataset/test/0 1200


In [16]:
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from sklearn import metrics

class Const:
    def __init__(self):
        self.path_trn='/kaggle/working/dataset/train'
        self.path_tst='/kaggle/working/dataset/test'
        self.path_out='/kaggle/working/out'
        self.path_model="{}/w.h5".format(self.path_out)

        self.sz=(128, 128)
        self.is_overfitting=False
        self.is_aug=True

        self.dropout=0.25
        self.lr=.01
        self.decay_factor=.5
        self.decay_min_lr=.00001

        self.epochs=100
        self.bsz_infer=100
        self.val_split=.2
        self.batch_sz=20
        self.estop_pateience=10


def stats(ctrn,class_weights,const):
    cls = np.unique(ctrn)
    ntotal = sum([len(os.listdir("{}/{}".format(const.path_trn, c))) for c in cls])
    ntrn = int(ntotal * (1 - const.val_split))
    nval = int(ntotal * const.val_split)
    ntst = sum([len(os.listdir("{}/{}".format(const.path_tst, c))) for c in cls])
    print("class weights", class_weights, "ntrain", ntrn, "nval", nval, \
          "ntst", ntst, "trn_left", ntrn % const.batch_sz, "tst_left", ntst % const.batch_sz)
    return ntrn,nval,ntst


def get_gtrn(path, sz, validation_split, is_overfitting=False, is_aug=True):
    if is_aug:
        data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip = True,
            validation_split=validation_split,
            rescale=1./255,
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.2
        )
    else:
        data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=validation_split)
    tgen = data_gen.flow_from_directory(path, target_size=sz, class_mode='categorical', subset='training')
    if is_overfitting:
        vgen=tgen
    else:
        vgen = data_gen.flow_from_directory(path, target_size=sz, class_mode='categorical',subset='validation')
    return tgen, vgen


def get_gtst(path, sz):
    data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    gen = data_gen.flow_from_directory(path, target_size=sz, class_mode='categorical')
    return gen


def load_model(sz,const):
    input = tf.keras.Input(shape=(sz[0], sz[1], 3))

    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32, activation="relu")(input)
#    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32, activation=None)(input)
    out=tf.keras.layers.BatchNormalization()(out)
#    out=tf.keras.layers.ReLU()(out)
    out=tf.keras.layers.MaxPooling2D()(out)
#    out=tf.keras.layers.Dropout(const.dropout)(out)

    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, activation="relu")(out)
#    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, activation=None)(out)
    out=tf.keras.layers.BatchNormalization()(out)
#    out=tf.keras.layers.ReLU()(out)
    out=tf.keras.layers.MaxPooling2D()(out)
#    out=tf.keras.layers.Dropout(const.dropout)(out)

    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, activation="relu")(out)
#    out=tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, activation=None)(out)
    out=tf.keras.layers.BatchNormalization()(out)
#    out=tf.keras.layers.ReLU()(out)
    out=tf.keras.layers.MaxPooling2D()(out)
#    out=tf.keras.layers.Dropout(const.dropout)(out)

    out=tf.keras.layers.Flatten()(out)
    out=tf.keras.layers.Dense(512, activation='relu')(out)
#    out=tf.keras.layers.Dense(512, activation=None)(out)
    out=tf.keras.layers.BatchNormalization()(out)
#    out=tf.keras.layers.ReLU()(out)
#    out=tf.keras.layers.Dropout(const.dropout)(out)
    out=tf.keras.layers.Dense(2, activation='softmax')(out)

    model=tf.keras.models.Model(inputs=input, outputs=out)
#    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    loss = tf.keras.losses.CategoricalCrossentropy()
#    model.compile(optimizer='sgd', loss=loss, metrics=['accuracy'])
    opt = tf.keras.optimizers.RMSprop(learning_rate=const.lr)
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

    model.summary()
    return model


def train(const):
#    tf.debugging.set_log_device_placement(True)

    print("\nloading model",flush=True)
    model = load_model(const.sz,const)

    # prepare and balance data loaders
    print("\nloading training data",flush=True)
    gtrn, gval = get_gtrn(const.path_trn,const.sz, validation_split=const.val_split, \
                               is_overfitting=const.is_overfitting, is_aug=const.is_aug)
    ctrn = gtrn.classes
    class_weights = sklearn.utils.compute_class_weight(class_weight='balanced', classes=np.unique(ctrn), y=ctrn)
    class_weights = dict(zip(np.unique(ctrn), class_weights))

    # data stats
    print("data:",end=" ",flush=True)
    print(const.path_trn)
    stats(ctrn,class_weights,const)

    # train
    print("\ntraining",flush=True)
    ncpu = os. cpu_count()
    print("ncpu",ncpu)
    estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=const.estop_pateience, \
                                           verbose=2, mode="min", restore_best_weights=True)
    decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, \
                                                 factor=const.decay_factor, min_lr=const.decay_min_lr)
    history = model.fit(gtrn, validation_data=gval, epochs=const.epochs, \
                        class_weight=class_weights, callbacks=[estop, decay],
                        use_multiprocessing=True,workers=ncpu)
    model.save_weights(const.path_model)
    
    # training history visualization
    print("visualizing training process",flush=True)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['trn', 'val'], loc='upper left')
    plt.savefig("{}/acc.jpg".format(const.path_out))
    plt.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss')
    plt.ylabel('loss')
    plt.xlabel('e')
    plt.legend(['trn', 'val'], loc='upper left')
    plt.savefig("{}/loss.jpg".format(const.path_out))
    plt.close()

    
def eval(const):
    model=load_model(const.sz,const)
    model.load_weights(const.path_model)
    gtst=get_gtst(const.path_tst,const.sz)
    test_scores = model.evaluate(x=gtst, verbose=2)
    print("loss",test_scores[0],"acc",test_scores[1])

    
def test(const):
    model = load_model(const.sz,const)
    model.load_weights(const.path_model)

    ds_test = tf.keras.utils.image_dataset_from_directory(
      const.path_tst if not const.is_overfitting else const.path_trn,
      seed=123,
      image_size=const.sz
    )

    normalization_layer = tf.keras.layers.Rescaling(1./255)
    normalized_ds = ds_test.map(lambda x, y: (normalization_layer(x), y))

    P = model.predict(normalized_ds,batch_size=const.bsz_infer)

    for c in [0, 1]:
        Y=None
        for _, y in ds_test:
            if Y is None:
                Y=y
            else:
                Y=np.concatenate([Y,y],axis=-1)
        Y=tf.keras.utils.to_categorical(Y)
        position=[np.argmax(oh) for oh in tf.keras.utils.to_categorical([0,1])]
        print("class",c,"predicted shape",P.shape,"labels shape",Y.shape,"position",position)

        pos,neg=0,0
        for i in range(Y.shape[0]):
            if Y[i,np.argmax(P[i,:])]==1:
                pos+=1
            else:
                neg+=1
        print("pos",pos,"neg",neg)
        
        fpr, rec, _ = metrics.roc_curve(Y[:, position[c]], P[:, position[c]], pos_label=1)
        print(Y.shape,P.shape,fpr.shape,rec.shape)
        s_=0
        for i in range(fpr.shape[0]):
            g=rec[i]-fpr[i]
            g=g if g>0 else -g
            s_+=g
            #print("rec {:.3f} fpr {:.3f} gap {:.3f}".format(rec[i],fpr[i],g))
        print("\n")
        for i in range(fpr.shape[0]):
            if i%10==0:
                print("rec {:.3f} fpr {:.3f} gap {:.3f}".format(rec[i],fpr[i],g))
        print("avg",s_/fpr.shape[0])
        rec,fpr=[],[]
        print("\nsample predictions: class",c, P[:10, :], Y[:10, :])
        # calculating manual roc f detailed tp/fp/tn/fn stats
        for th in [1./(th+1) for th in range(10)]:
            data={'tp':0, 'fp':0, 'tn':0, 'fn':0}
            for i in range(P.shape[0]):
                if Y[i, position[c]]==1:
                    if P[i, position[c]]>=th:
                        data['tp']+=1
                    else:
                        data['fn']+=1
                else:
                    if P[i, position[c]]>=th:
                        data['fp']+=1
                    else:
                        data['tn']+=1
            pos = data['tp'] + data['fn']
            neg = data['tn'] + data['fp']

            rec += [float(data['tp'])/pos]
            fpr += [float(data['fp'])/neg]
            print("class {} th {:.2f} rec {:.3f} fpr {:.3f} tp {:.3f} fp {:.3f} tn {:.3f} fn {:.3f}".format(c,th,rec[-1],fpr[-1],data['tp'],data['fp'],data['tn'],data['fn']))
        plt.plot(fpr,rec)
        plt.title("{} roc".format(c))
        plt.ylabel('rec')
        plt.xlabel('fpr')
        plt.savefig("{}/roc{}.jpg".format(const.path_out,c))
        plt.close()


if __name__=='__main__':
    """
    start: subset , im size 32x32 , overfit (im size 32x32) , rmsprop
    res: trn loss plateaue
    """
    expr_name = "e10_sz32_drop0_bn0_aug0_overfit1_ntrn1K"
    c = Const()
    c.path_trn = "/kaggle/working/dataset/train1000"
    c.epochs = 10
    c.dropout = 0
    c.is_overfitting = True
    c.is_aug = False
    c.sz = (32, 32)
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)

    """
    changes: all data
    res: trn loss plateaue
    """
    expr_name = "e50_sz32_drop0_bn0_aug0_overfit1_ntrnALL"
    c = Const()
    c.path_trn = "/kaggle/working/dataset/train"
    c.epochs = 50
    c.dropout = 0
    c.is_overfitting = True
    c.is_aug = False
    c.sz = (32, 32)
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)

    """
    changes: im size 128x128 , non overfit , aug
    res: trn loss plateaue
    """
    expr_name = "e100_drop0_bn1_aug1_overfit0_ntrnALL"
    c = Const()
    c.path_train = "/kaggle/working/dataset/train"
    c.epochs = 100
    c.dropout = 0
    c.is_overfitting = False
    c.is_aug = True
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)
    
    """
    changes: overfit , dropout p=.25
    res: trn loss plateaue
    """
    expr_name = "e50_sz128_drop25_bn0_aug1_overfit1_ntrnALL"
    c = Const()
    c.path_train = "/kaggle/working/dataset/train"
    c.epochs = 50
    c.dropout = .25
    c.is_overfitting = True
    c.is_aug = True
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)

    """
    prev: e100_drop0_bn1_aug1_overfit0_ntrnALL
    changes: overfit , bn
    res: trn & val losses converges in 15e
    nxt: restart & check trn roc after 15e
    """
    expr_name = "e50_sz128_drop0_bn1_aug1_overfit1_ntrnALL"
    c = Const()
    c.path_train = "/kaggle/working/dataset/train"
    c.epochs = 50
    c.dropout = 0
    c.is_overfitting = True
    c.is_aug = True
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)
    #test(c)

    """
    changes: 15 epochs
    res: TBD
    """
    expr_name = "e15_drop0_bn1_aug1_overfit1_ntrnALL"
    c = Const()
    c.path_train = "/kaggle/working/dataset/train"
    c.epochs = 15
    c.dropout = 0
    c.is_overfitting = True
    c.is_aug = True
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    print(expr_name)
    #train(c)
    #eval(c)
    test(c)
    
    
    # all data , im size 128x128 , non overfit , bn , drop .25 , aug : trn loss pass > val loss pass > tst roc fail
    expr_name = "e100_drop25_bn1_aug1_overfit0_ntrnALL"
    c = Const()
    c.path_train = "/kaggle/working/dataset/train"
    c.epochs = 100
    c.dropout = .25
    c.is_overfitting = False
    c.is_aug = True
    c.path_model = "{}/w_{}.h5".format(c.path_out, expr_name)
    #print(expr_name)
    #train(c)
    #test(c)

e15_drop0_bn1_aug1_overfit1_ntrnALL
Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_92 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_93 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_70 (MaxP